In [2]:
%matplotlib inline
#Amitava Chakraborty - 16th Feb 2020
# Swap Test - Given two unknown quantum states, determine how much them differ

# Importing standard Qiskit libraries and configuring account
import numpy as np
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute
from qiskit import IBMQ, BasicAer
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity
from qiskit.visualization import plot_histogram

q_simulator = BasicAer.get_backend('qasm_simulator')
s_simulator = BasicAer.get_backend('statevector_simulator')

In [3]:
qr = QuantumRegister(3)  # Initialize qubits
cr = ClassicalRegister(1)  # Initialize bits for record measurements
circuit = QuantumCircuit(qr, cr)

# Create 'unknown' states
circuit.x(qr[1])
circuit.barrier()

# Swap test
circuit.h(qr[2])
circuit.cswap(qr[2], qr[0], qr[1])
#circuit.h(qr[2])

circuit.barrier()

# Measure
circuit.measure(qr[2], cr[0])
# If unknown states are: 
# 1. Orthogonal, then 0 is measured with probability 50%
# 2. Equal, then 0 is measured with probability 100%

In [4]:
# Run the circuit with local simulator
results = execute(circuit, backend=q_simulator, shots=1024).result()
answer = results.get_counts()
print(answer)
# Since our states are orthogonal (qr[0] = |0>, qr[1] = |1>)
# 0 is observed with probability approx. 50%  and
# 1 is observed with probability approx. 50%.

{'1': 518, '0': 506}


Say, $\left\vert \psi \right\rangle$ can be either of two values, a or b. Can we check the value of $\left\vert \psi \right\rangle$?

We may check for state equality with the SWAP test.
[Quantum fingerprinting (Buhrman, Cleve, Watrous & de Wolf, 2001)](https://arxiv.org/abs/quant-ph/0102001) seems to be the first paper to introduce the SWAP test.

The idea behind this test is:

 1. Encode the 2 quantum states using quantum error correction codes to "increase the difference between them".
 2. Test the 2 code words by using an ancilla register and the procedure below.
 3. Read the ancilla register. If it is $\left\vert 0 \right\rangle$ then the 2 states are probably equal. Else, they are probably different. 

<img src='images/swapTest.png' alt="" width="500" align="middle"/>

We can repeat the procedure multiple times to ensure that the 2 states are equal up to a given probability.

In [8]:
register_size = 2

qr_psi = QuantumRegister(register_size, 'psi')  #For state PSI
qr_phi = QuantumRegister(register_size, 'phi')  #For state PHI
qr_ancilla = QuantumRegister(1, 'ancilla')
cequal = ClassicalRegister(1, 'equal')

circuit = QuantumCircuit()

circuit.add_register(qr_psi)
circuit.add_register(qr_phi)
circuit.add_register(qr_ancilla)
circuit.add_register(cequal)


def cswap(circuit, ctrl, q1, q2) -> None:
    assert(len(q1) == len(q2), "The swapped register sizes should match")
    for i in range(len(q1)):
        # Controlled swap
        circuit.ccx(ctrl, q1[i], q2[i])
        circuit.ccx(ctrl, q2[i], q1[i])
        circuit.ccx(ctrl, q1[i], q2[i])

def equality_test(circuit, ancilla, q1, q2, classical_register) -> None:
    assert(len(q1) == len(q2), "The swapped register sizes should match")
    circuit.h(ancilla[0])
    cswap(circuit, ancilla[0], q1, q2)
    circuit.h(ancilla[0])
    circuit.measure(ancilla[0], classical_register[0])

## Initialisation
# We add Hadamard to all the registers - to create PSI
circuit.x(qr_phi)
circuit.h(qr_psi)
# We add Hadamard to all the registers - to create PHI
# Modify the initialisation of either PHI or PSI and check the results.
circuit.y(qr_phi)
circuit.h(qr_phi)

## SWAP test
equality_test(circuit, qr_ancilla, qr_psi, qr_phi, cequal)

res_qasm = execute([circuit], q_simulator, shots=1024).result()
counts = res_qasm.get_counts()

print(counts)

{'0': 1024}


<>:17: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:25: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:17: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:25: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-8-68e663ef8b0f>:17: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(q1) == len(q2), "The swapped register sizes should match")
<ipython-input-8-68e663ef8b0f>:25: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(q1) == len(q2), "The swapped register sizes should match")




[1]: images/swapTest.png